In [1]:
import sys
sys.path.insert(0,'/home/kyle/GWA/NANOGrav/PsrSigSim/')
import psrsigsim as PSS
import numpy as np
import h5py


#Bokeh imports
from bokeh.io import curdoc, output_file, show
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource, Range1d
import bokeh.models.widgets as widgets
from bokeh.plotting import figure

In [2]:
#Default values for psr_dict----------------------------------------------------
psr_dict = {}
psr_dict['f0'] = 1400                   #Central frequency
psr_dict['F0'] = 218                    #Pulsar spin freq
psr_dict['bw'] = 400                    #Bandwidth
psr_dict['Nf'] = 512                    #Frequency bins
psr_dict['ObsTime'] = 20                #Observation time
psr_dict['f_samp'] = 4                  #Sampling frequency
psr_dict['SignalType'] = "intensity"    #'intensity' which carries a Nf x Nt
#filterbank of pulses or 'voltage' which carries a 4 x Nt array of
#voltage vs. time pulses representing 4 stokes channels
psr_dict['dm'] = 0.1                     #Dispersion Measure Pescs/(CM^3)
# V_ISS -- Intersteller Scintilation Velocity
psr_dict['scint_bw'] =  15.6            #Scintilation Bandwidth
psr_dict['scint_timescale'] = 2630      #Scintilation Timescale
# pulsar -- pulsar name
# telescope -- telescope name(GBT or Arecibo)
psr_dict['freq_band'] = 1400            #Frequency band [327 ,430, 820, 1400, 2300]
# aperature -- aperature (m)
# area -- collecting area (m^2)
# Tsys -- system temp (K), total of receiver, sky, spillover, etc. (only needed for noise)
# name -- GBT or Arecibo
# tau_scatter -- scattering time (ms)
psr_dict['radiometer_noise'] =  False   #radiometer noise
psr_dict['data_type']='float32'            #
psr_dict['flux'] = 3
psr_dict['to_DM_Broaden'] = True

#Constants for generating data--------------------------------------------------
dm_range = (0.1,10.1)
dm_range_spacing = 5
NumPulses = 1
startingPeriod = 1.0
start_time = (startingPeriod / psr_dict['F0']) *1000  #Getting start time in ms
TimeBinSize = 0.0002499958334027766
start_bin = int((start_time)/TimeBinSize)
stop_time = (((1 / psr_dict['F0']) *1000) * NumPulses) + start_time
# start_time + however many pulses times the pulsar period in ms
stop_bin =int((stop_time)/TimeBinSize)


In [3]:
fig = figure()

dmSlider = widgets.Slider(title="Dispersion Measure", value= 0.1,
                          start=dm_range[0], end=dm_range[1],
                          step=dm_range_spacing)

Exbutton = widgets.Button(label='Generate Filter Bank', button_type='success')

In [6]:
FullData = []
i = 0
while i<=10:
    psr_dict['dm']=i
    psr = PSS.Simulation(psr =  'J1713+0747' , sim_telescope= 'GBT',sim_ism= True, sim_scint= False, sim_dict = psr_dict)
    psr.simulate()
    FullData.append(psr.signal.signal[:,start_bin:stop_bin])
    i+=10

INFO: Using ICRS equatorial coordinate. The parameter KOM is measured respect to equatorial North. [pint.models.binary_ddk]
Five warnings should appear. Warnings that are normal are: Unrecognized parfile line "SOLARN0", "T2CMETHOD", "MODE", "INFO -f". Warning about "DDK model" is also normal.  
Two 'F0' values input. Simulator will default to input dictionary value.
Two 'dm' values input. Simulator will default to input dictionary value.


/home/kyle/GWA/NANOGrav/PsrSigSim/psrsigsim/ism.py:237: RuntimeWarning: invalid value encountered in true_divide
  input_array_norm = input_array[ii,:] / input_array_sum
/home/kyle/anaconda3/lib/python3.5/site-packages/mkl_fft/_numpy_fft.py:1044: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfftn_numpy(a, s, axes)


98% dispersed in 3.969 seconds.

INFO: Using ICRS equatorial coordinate. The parameter KOM is measured respect to equatorial North. [pint.models.binary_ddk]
Five warnings should appear. Warnings that are normal are: Unrecognized parfile line "SOLARN0", "T2CMETHOD", "MODE", "INFO -f". Warning about "DDK model" is also normal.  
Two 'F0' values input. Simulator will default to input dictionary value.
Two 'dm' values input. Simulator will default to input dictionary value.
98% dispersed in 6.285 seconds.

In [37]:
num = 0
source = ColumnDataSource(data=dict(image=FullData[num][:,:]))

In [12]:
array1 = [[0,1,2],
          [3,4,5],
          [6,7,8]]
array2 = [[9,10,11],
          [12,13,14],
          [15,16,17]]
array3 = [[18,19,20],
          [21,22,23],
          [24,25,26]]

totarray= [array1,array2,array3]

In [27]:
test= np.array(FullData)

In [28]:
print(type(test))

<class 'numpy.ndarray'>


In [6]:
psr.signal.last_freq

1600.0

In [ ]:
f = h5py.File('NewTestFile.hdf5','w')
dataString = 'Data'
f.create_dataset(dataString, data=FullData)
f.close()

In [ ]:
FullData = None
print(FullData)

In [ ]:
f = h5py.File('NewTestFile.hdf5','r')
dataString = 'Data'
FullData = np.array(f.get(dataString))
f.close()

In [ ]:
print(FullData[1])

In [35]:
N = 100

x_ = np.linspace(0, 10, 200)
y_ = np.linspace(0, 10, 200)
z_ = np.linspace(0, 10, N)

x, y, z = np.meshgrid(x_, y_, z_, indexing='xy')

data = np.sin(x+z)*np.cos(y)

source = ColumnDataSource(data=dict(image=[data[:, :, 0]]))

